# Neccessary Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from Data_Splitter import split_users, holdout_interactions
from Fairness_Metrics import compute_recGap, compute_compounding_factor

In [2]:
df = pd.read_csv('data/LFM-1b-DemoBiasSub-10k.csv', header=0)
df

,user_id,artist_id,track_id,play_count,country,age,gender,registered_unixtime
0,384,362,16567,16,UK,35,m,1035849600
1,384,2000,12303,9,UK,35,m,1035849600
2,384,2000,12308,40,UK,35,m,1035849600
3,384,2583,151308,15,UK,35,m,1035849600
4,384,2583,151314,43,UK,35,m,1035849600
...,...,...,...,...,...,...,...,...
350185,50871714,1893,1606576,7,BY,19,f,1342728447
350186,50871714,12189,992547,5,BY,19,f,1342728447
350187,50871714,48241,791386,5,BY,19,f,1342728447
350188,50871714,49735,3561201,5,BY,19,f,1342728447


# Data Splitter

In [5]:
# Build the data

# Split users into train/validation/test groups (based solely on user_id)
train_users, val_users, test_users = split_users(df, train_frac=0.6, val_frac=0.2, test_frac=0.2)

# For train users, use all interactions.
df_train_all = df[df['user_id'].isin(train_users)]

# For validation and test users, further split each user's interactions (80% train, 20% holdout)
df_val_all = df[df['user_id'].isin(val_users)]
df_test_all = df[df['user_id'].isin(test_users)]

df_val_train, df_val_holdout = holdout_interactions(df_val_all, holdout_frac=0.2)
df_test_train, df_test_holdout = holdout_interactions(df_test_all, holdout_frac=0.2)

# Build the overall training data by combining all train users and the training portion for validation/test users.
df_model_train = pd.concat([df_train_all, df_val_train, df_test_train]).reset_index(drop=True)

# KNNItem Build

In [ ]:
# Create a binarized user–track interaction matrix from the training data.
user_track_matrix = pd.crosstab(df_model_train['user_id'], df_model_train['track_id'])
user_track_matrix = (user_track_matrix > 0).astype(int)

# For an item-based KNN recommender, we treat each track (item) as a vector of user interactions.
# Transpose the matrix and create a sparse representation (tracks x users).
sparse_item_matrix = csr_matrix(user_track_matrix.T.values)

# Train a KNN model on the item (track) vectors.
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(sparse_item_matrix)

# Prepare a list of track IDs (corresponding to the columns in the user_track_matrix)
track_list = list(user_track_matrix.columns)

In [ ]:
def get_item_similarities(track_id, n_neighbors=5):
    """
    Retrieve the n most similar tracks for a given track_id.
    """
    if track_id not in track_list:
        return []
    track_index = track_list.index(track_id)
    track_vector = sparse_item_matrix[track_index]
    # Retrieve neighbors (n_neighbors + 1 because the track itself is returned)
    distances, indices = knn_model.kneighbors(track_vector, n_neighbors=n_neighbors + 1)
    # Skip the first index if it is the track itself.
    similar_indices = [i for i in indices.flatten() if i != track_index]
    similar_tracks = [track_list[i] for i in similar_indices]
    return similar_tracks

def get_recommendations_for_user(user_id, top_n=10, n_neighbors=10):
    """
    For a given user, aggregate similar items from the items the user has interacted with in the training data.
    Only recommend items the user has not interacted with.
    """
    if user_id not in user_track_matrix.index:
        return []
    # Get the set of tracks the user has interacted with (training interactions)
    user_history = set(user_track_matrix.loc[user_id][lambda row: row == 1].index)
    
    candidate_scores = {}
    # For each item in the user history, get similar items and sum a simple frequency score.
    for item in user_history:
        similar_items = get_item_similarities(item, n_neighbors=n_neighbors)
        for sim_item in similar_items:
            if sim_item in user_history:
                continue
            candidate_scores[sim_item] = candidate_scores.get(sim_item, 0) + 1
                
    ranked_items = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_items = [item for item, score in ranked_items][:top_n]
    return recommended_items

## KNNItem Evaluation Using NDCG

In [ ]:
def ndcg_at_k(relevances, k):
    """
    Compute NDCG@k given a list of binary relevance scores.
    """
    relevances = np.asfarray(relevances)[:k]
    if relevances.size == 0:
        return 0.0
    # Discount factors (log2-based)
    discounts = np.log2(np.arange(2, relevances.size + 2))
    dcg = np.sum(relevances / discounts)
    # Ideal DCG (sorted relevances)
    ideal_relevances = np.sort(relevances)[::-1]
    idcg = np.sum(ideal_relevances / discounts)
    return dcg / idcg if idcg > 0 else 0.0

def evaluate_ndcg(holdout_df, top_n=10, n_neighbors=10):
    """
    Evaluate recommendations using NDCG@k for each user in a holdout set.
    Returns overall NDCG and NDCG by gender.
    """
    # Create mapping from user_id to their holdout (ground truth) track_ids.
    user_holdout = holdout_df.groupby('user_id')['track_id'].apply(set).to_dict()
    # Get user genders from the original data (assuming 'gender' column exists).
    user_gender = df.set_index('user_id')['gender'].to_dict()
    
    ndcg_scores = {}    # per user scores
    ndcg_by_gender = {} # aggregated scores per gender
    
    for user, true_items in user_holdout.items():
        # Generate recommendations using the training data.
        recs = get_recommendations_for_user(user, top_n=top_n, n_neighbors=n_neighbors)
        # Binary relevance: 1 if the recommended item is in the holdout set, 0 otherwise.
        relevances = [1 if rec in true_items else 0 for rec in recs]
        ndcg = ndcg_at_k(relevances, top_n)
        ndcg_scores[user] = ndcg
        
        gender = user_gender.get(user, 'unknown')
        if gender not in ndcg_by_gender:
            ndcg_by_gender[gender] = []
        ndcg_by_gender[gender].append(ndcg)
    
    overall_ndcg = np.mean(list(ndcg_scores.values())) if ndcg_scores else 0.0
    avg_ndcg_by_gender = {gender: np.mean(scores) for gender, scores in ndcg_by_gender.items()}
    return overall_ndcg, avg_ndcg_by_gender


def grid_search_validation(val_holdout_df, candidate_neighbors, candidate_top_n):
    """
    Perform grid search over n_neighbors and top_n parameters on the validation holdout set.
    Returns the best hyperparameters (those that achieve the highest overall NDCG) and grid search results.
    """
    best_ndcg = -1.0
    best_params = None
    grid_results = []  # Store tuples: (n_neighbors, top_n, overall_ndcg)
    
    for n_neighbors_param in candidate_neighbors:
        for top_n_param in candidate_top_n:
            overall_ndcg_val, _ = evaluate_ndcg(val_holdout_df, top_n=top_n_param, n_neighbors=n_neighbors_param)
            grid_results.append((n_neighbors_param, top_n_param, overall_ndcg_val))
            print(f"n_neighbors: {n_neighbors_param}, top_n: {top_n_param} => NDCG: {overall_ndcg_val:.4f}")
            if overall_ndcg_val > best_ndcg:
                best_ndcg = overall_ndcg_val
                best_params = (n_neighbors_param, top_n_param)
    
    return best_params, best_ndcg, grid_results

In [ ]:
# Define candidate hyperparameters.
candidate_neighbors = [5, 10, 15]
candidate_top_n = [10]

best_params, best_ndcg, grid_results = grid_search_validation(df_val_holdout, candidate_neighbors, candidate_top_n)
print("\nBest hyperparameters (n_neighbors, top_n):", best_params)
print("Best overall NDCG on validation set:", best_ndcg)


# Evaluate on the validation set
if best_params is not None:
    best_n_neighbors, best_top_n = best_params
else:
    # Default in case grid search fails.
    best_n_neighbors, best_top_n = 10, 10

overall_ndcg_test, ndcg_by_gender_test = evaluate_ndcg(df_test_holdout, top_n=best_top_n, n_neighbors=best_n_neighbors)
print("\nTest Set Evaluation:")
print(f"Overall NDCG@{best_top_n}: {overall_ndcg_test:.4f}")
print("NDCG by gender on test set:", ndcg_by_gender_test)

n_neighbors: 5, top_n: 10 => NDCG: 0.1665
n_neighbors: 10, top_n: 10 => NDCG: 0.1613
n_neighbors: 15, top_n: 10 => NDCG: 0.1591

Best hyperparameters (n_neighbors, top_n): (5, 10)
Best overall NDCG on validation set: 0.1664797411405441

Test Set Evaluation:
Overall NDCG@10: 0.1624
NDCG by gender on test set: {'m': 0.16615608516935818, 'f': 0.15200149805676685}


# Fairness Evaluation

In [4]:
compute_recGap(ndcg_by_gender_test)

RecGap Score: 0.014154587112591321


In [5]:
compute_compounding_factor(df, ndcg_by_gender_test)

Original data distribution:
Males: 0.7575144921328422
Females: 0.24248550786715783

Metric score distribution:
Males: 0.7734925082231681
Females: 0.2265074917768319

Compounding Factor (KL Divergence): 0.0007169252234241787
